# ACCESS-NRI 2025 -- CMWG Workshop 

# Visualising ISSM Models - *Part 1*

`pyISSM` has a series of efficient visualisation tools, accessible via the `pyissm.plot` submodule. Key visualisation tools currently include:
- Mesh visualisation
- Boundary conditions
- 2D field visualisation (input and results)
- Transient results (1D and 2D)

All pyISSM visualisation capabilities leverage existing Python modules (e.g. `matplotlib` and `xarray`). Users can create complete complex and publication-ready figures using familiar syntax. By default, all `pyISSM` visualisation functions return a figure and set of axes, consistent with `matplotlib.pyplot.subplots()`. Optionally, an existing axes object can be passed to the `pyISSM` function for more refined control of plotting.

<div class="alert alert-block alert-info">
<b>AIM:</b> This notebook focuses on visualising ISSM model meshes, element/node types, and model boundary conditions using pyISSM.
</div>

In [ ]:
# Import required modules
import matplotlib.pyplot as plt
import matplotlib.patches

# Import pyISSM
# NOTE: Here, we assume that the `pyISSM` git repository has been cloned onto your machine. If `pyISSM` has been installed using `conda`, skip the `sys` calls.
import sys
import socket
hostname = socket.gethostname()
if 'gadi' in hostname:
    sys.path.append('/home/565/lb9857/gitRepos/pyISSM/src/')
else:
    sys.path.append('/Users/lawrence.bird/pyISSM/src/')
    import matplotlib
    matplotlib.use('MacOSX')
    %matplotlib inline
import pyissm

In [ ]:
# Load an existing ISSM model onto md

if 'gadi' in hostname:
    md = pyissm.model.io.load_model('/home/565/lb9857/gitRepos/CMWG_workshop_2025/sampleModels/Greenland.HistoricTransient_200yr.nc')
else:
    md = pyissm.model.io.load_model('/Users/lawrence.bird/access-dev/sampleModels/Greenland.HistoricTransient_200yr.nc')

## 1. Visualise the model mesh

ISSM uses a triangular mesh that can include anistropy. To visualise this, `pyISSM` leverages the `matplotlib.pyplot.triplot` library.

Before visualising the mesh, we need to process the model to generate a compatible trinagulation object. We can efficiently extract all required components of the mesh using `pyissm.model.mesh.process_mesh()`.

Below, let's process the mesh and make a simple plot of the model mesh.

<div class="alert alert-block alert-warning">
<b>3D Models:</b> ISSM supports computation of 3D models. However, pyISSM does not yet support full-visualisation of 3D models. When visualising a 3D model, pyISSM will display the 2D projection of the mesh by default.
</div>

In [ ]:
# Extract the mesh information from the model, creating the necessary matplotlib.tri.triangulation.Triangulation object
mesh, mesh_x, mesh_y, mesh_elements, is3d = pyissm.model.mesh.process_mesh(md)

# Visualise the model mesh (pyISSM default)
fig, ax = pyissm.plot.plot_mesh2d(mesh, figsize = (4, 6))

By default, `pyISSM` returns a Figure and Axis object, visualising the mesh, and assigns X and Y labels. We can provide custom X/Y labels directly to the `pyissm.plot.plot.mesh_2d()` function. All components of these Figure and Axis objects can be altered using standard `matplotlib` functionality.

For example, we can modify the axes labels, tick marks/units, and add a title as follows:

In [ ]:
# Visualise the model mesh (pyISSM default)
fig, ax = pyissm.plot.plot_mesh2d(mesh, figsize = (4, 6),
                                  xlabel = 'Model X coordinate (km)',
                                  ylabel = 'Model Y coordinate (km)')

# Adjust the X/Y ticks
ax.set_xticks([-5*1e5, 0, 5*1e5])
ax.set_xticklabels(['-500', '0', '500'])
ax.set_yticks([-1*1e6, -2*1e6, -3*1e6])
ax.set_yticklabels(['-1000', '-2000', '-3000'])

# Add title
plt.title('Model mesh')

plt.show()

We can add additional complexity to this plot by passing additional arguments to the `pyissm.plot.plot_mesh2d()` command. Below, we generate a 3 panel plot, each with additional complexity. Comments in the below cell provide a description of the functionality added to each subplot.

In [ ]:
# Generate a 3-panel figure, with shared X/Y axes, constrained layout, and custom figure size
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (12,6), sharex = True, sharey = True, constrained_layout = True)

## Panel 1
# Add 'nodes' to show the X/Y vertices defined in md.mesh.x and md.mesh.y. Provide empty xlabel to turn this off.
ax1 = pyissm.plot.plot_mesh2d(mesh,
                              ax = ax1,
                              show_nodes = True,
                              ylabel = 'Model Y coordinate (km)',
                              xlabel = '')

## Panel 2
# Change the color and thickness of the mesh lines. Provide an empty ylabel to turn this off.
ax2 = pyissm.plot.plot_mesh2d(mesh,
                              ax = ax2,
                              show_nodes = True,
                              color = 'red',
                              linewidth = 1.1,
                              ylabel = '',
                              xlabel = 'Model X coordinate (km)')

## Panel 3
# Change the color, shape, and size of the nodes. Provide an empty xlabel and ylabel to turn these off.
ax3 = pyissm.plot.plot_mesh2d(mesh,
                              ax = ax3,
                              show_nodes = True,
                              color = 'red',
                              linewidth = 1.1,
                              node_args = {'color': 'blue',
                                           'marker': '*',
                                           's': 20},
                              ylabel = '',
                              xlabel = '')

# Adjust the X/Y ticks (only need to adjust ax1 as all subplots share X/Y axes)
ax1.set_xticks([-5*1e5, 0, 5*1e5])
ax1.set_xticklabels(['-500', '0', '500'])
ax1.set_yticks([-1*1e6, -2*1e6, -3*1e6])
ax1.set_yticklabels(['-1000', '-2000', '-3000'])

plt.show()

Because `pyISSM` returns `matplotlib` Figure and Axis objects, it's easy to add additional detail to the plots. Let's create a 2-panel plot with a zoomed region.

In [ ]:
# Define a region for the Zoom (xmin, xmax, ymin, ymax)
zoom_region = [2*1e5, 6.5*1e5, -2.5*1e6, -2*1e6]

# Create a Rectangle object
rect = matplotlib.patches.Rectangle((zoom_region[0], zoom_region[2]),
                                    zoom_region[1] - zoom_region[0],
                                    zoom_region[3] - zoom_region[2],
                                    linewidth = 1,
                                    edgecolor = 'red',
                                    facecolor = 'none')

# Create 2-panel subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10,7), constrained_layout = True)

# Add default pyISSM mesh plot to ax1 and add Rectangle object
ax1 = pyissm.plot.plot_mesh2d(mesh,
                              ax = ax1,
                              ylabel = 'Model Y coordinate (km)',
                              xlabel = 'Model X coordinate (km)')
ax1.add_patch(rect)

# Add customised mesh plot to ax2
ax2 = pyissm.plot.plot_mesh2d(mesh,
                              ax = ax2,
                              show_nodes = True,
                              node_args = {'color': 'red',
                                           's': 20},
                              ylabel = '',
                              xlabel = 'Model X coordinate (km)')

# Adjust the X/Y limits of ax2
ax2.set_xlim(zoom_region[0], zoom_region[1])
ax2.set_ylim(zoom_region[2], zoom_region[3])

# Adjust X/Y ticks on ax1 and ax2
ax1.set_xticks([-5*1e5, 0, 5*1e5])
ax1.set_xticklabels(['-500', '0', '500'])
ax1.set_yticks([-1*1e6, -2*1e6, -3*1e6])
ax1.set_yticklabels(['-1000', '-2000', '-3000'])

ax2.set_xticks([2*1e5, 4*1e5, 6*1e5])
ax2.set_xticklabels(['200', '400', '600'])
ax2.set_yticks([-2*1e6, -2.2*1e6, -2.4*1e6])
ax2.set_yticklabels(['-2000', '-2200', '-2400'])

plt.show()

# 2. Visualise model node and element types

ISSM models use two `mask` objects to define the presence of ice and ocean:
- `md.mask.ice_levelset`: presence of ice if < 0, icefront position if = 0, no ice if > 0
- `md.mask.ocean_levelset`: presence of ocean if < 0, coastline/grounding line if = 0, no ocean if > 0

Leveraging these masks, we can identify different node/element types, including the ice-front and grounding line positions. In `pyISSM`, we can interogate different model element types using `pyissm.plot.plot_model_elements()` and different node types using `pyissm.plot.plot_model_nodes()`.

Below, let's plot different element and node types for this model.

## 2.1 Model element types

During model development, it's important to ensure that particularly the ice front is correctly defined as this has implications on boundary conditions within the model. `pyissm.plot.plot_model_elements()` can display various elements types:
- `ice_elements`: All elements that are ice covered (grounded or floating)
- `ice_front_elements`: All elements that are along the ice front
- `ocean_elements`: All elements that contain ocean (either open ocean or floating ice)
- `floating_ice_elements`: All elements that contain floating ice
- `grounded_ice_elements`: All elements that contain grounded ice
- `grounding_line_elements`: All elements that are along the grounding line

Below, let's visualise all elements that are ice covered and those along the ice front.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = True, sharey = True, figsize = (10, 7), constrained_layout = True)

# By default, 'ice_elements' are plotted.
ax1 = pyissm.plot.plot_model_elements(md,
                                      md.mask.ice_levelset,
                                      md.mask.ocean_levelset,
                                      ax = ax1)

# Show elements on the ice-front.
ax2 = pyissm.plot.plot_model_elements(md,
                                      md.mask.ice_levelset,
                                      md.mask.ocean_levelset,
                                      ax = ax2,
                                      type = 'ice_front_elements',
                                      legend_args = {'loc': 'lower right'})

plt.show()

## 2.2 Model node types

Since `md.mask` objects are defined on vertices, it can be useful to visualise similar information to the elements above, but for the model vertices. `pyissm.plot.plot_model_nodes()` can display the following node types:
- `ice_nodes`: All nodes defined as ice
- `ice_front_nodes`: Any nodes explicitly defined as the ice front
- `ocean_nodes`: All nodes that contain ocean (either open ocean or floating ice)
- `floating_ice_nodes`: All nodes that contain floating ice
- `grounded_ice_nodes`: All nodes that contain grounded ice

Below, let's visualise all nodes that are ice covered and those along the ice front.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex = True, sharey = True, figsize = (10, 7), constrained_layout = True)

# By default, 'ice_elements' are plotted.
ax1 = pyissm.plot.plot_model_nodes(md,
                                   md.mask.ice_levelset,
                                   md.mask.ocean_levelset,
                                   ax = ax1)

# Show elements on the ice-front.
ax2 = pyissm.plot.plot_model_nodes(md,
                                   md.mask.ice_levelset,
                                   md.mask.ocean_levelset,
                                   ax = ax2,
                                   type = 'ice_front_nodes',
                                   legend_args = {'loc': 'lower right'})

plt.show()

## 3. Model boundary conditions

Correctly defining boundary conditions within ISSM is critical to providing reliable model results. Boundary conditions are generally defined using parameters prepended with `spc` (e.g. `md.stressbalance.spcvx` defines x-component velocity constraints for the stressbalance). `pyissm.plot.plot_model_bc()` provides a convenient tool to visualise such boundary conditions.

Below, let's plot the boundary conditions for the stressbalance solution in ISSM.

Boundary conditions/constraints for other solutions can be plotted by providing the following types (solutions):
- `stressbalance`: Stress balance boundary conditions (Default)
- `masstransport`: Mass transport constraints
- `thermal`: Thermal constraints
- `balancethickness`: Balance thickness constraints
- `debris`: Debris constraints
- `levelset`: Levelset constraints

In [ ]:
fig, ax = pyissm.plot.plot_model_bc(md,
                                    figsize = (5, 7),
                                    legend_args = {'loc': 'lower right'})

plt.show()